**Notebook 2** (v2)

Ingesta automatizada y creación de Bronze Layer

Objetivos del presente notebook:
- Automatizar la descarga de datos
- Estandarizar el proceso de ingesta
- Validación inicial robusta

Versiones:
- v2: limpieza de v1; acabado

**Librerías**

In [1]:
import pandas as pd

from pyspark.sql import SparkSession, DataFrame

#from pyspark.sql.functions import * -> no usar para evitar problemas de compatibilidad Python-Spark
from pyspark.sql.functions import (
    sum as spark_sum,
    min as spark_min,
    max as spark_max,
    col, when, current_timestamp, lit
)

from pyspark.sql.types import *
from pyspark.sql import functions as F

from datetime import datetime, timedelta
from typing import List, Dict, Optional, Any

import requests

from urllib.parse import urlparse
from google.colab import drive

import os
import json
import hashlib

**Google Drive**

In [2]:
# monta Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# crea la estructura de directorios

# configuración de paths
PROJECT_ROOT = "/content/drive/MyDrive/taxi_project"
RAW_DIR = f"{PROJECT_ROOT}/raw"
BRONZE_DIR = f"{PROJECT_ROOT}/bronze"
METADATA_DIR = f"{PROJECT_ROOT}/metadata"

# ruta donde guardar la capa Bronze
BRONZE_PATH = f"{BRONZE_DIR}/taxi_data"

# crea directorios si no existen
for path in [RAW_DIR, BRONZE_DIR, METADATA_DIR]:
    os.makedirs(path, exist_ok=True)

**Diagrama de carpetas y archivos**

Sigue la estructura de particiones de Hive.

/content/drive/MyDrive/taxi_project/<br>
├── raw/ ## archivos originales descargados<br>
│   ├── yellow_tripdata_2023-01.parquet<br>
│   ├── yellow_tripdata_2023-02.parquet<br>
│   └── yellow_tripdata_2023-03.parquet<br>
│<br>
├── bronze/ ## datasets validados en formato Parquet<br>
│   └── taxi_data/<br>
│       ├── ingestion_year=2023/<br>
│       │   ├── ingestion_month=01/<br>
│       │   │   └── part-*.snappy.parquet<br>
│       │   ├── ingestion_month=02/<br>
│       │   │   └── part-*.snappy.parquet<br>
│       │   └── ingestion_month=03/<br>
│       │       └── part-*.snappy.parquet<br>
│<br>
├── metadata/ ## almacén de logs y metadatos de ingesta<br>
│   ├── ingestion_log.jsonl ## logs de eventos de ingesta<br>
│   └── bronze_layer_metadata.json ## metadatos de la capa Bronze

**Configuración de Spark**

In [4]:
# setup para Spark en Google Colab

# instala Java si no está
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null

# fija JAVA_HOME
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

# asegura versión compatible de PySpark
!pip install -q pyspark==3.5.1

In [5]:
# configuración optimizada de Spark para Colab
spark = SparkSession.builder \
    .appName("NYC-Taxi-Ingesta") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

print(f"✅ Spark inicializado - Version: {spark.version}")

✅ Spark inicializado - Version: 3.5.1


**Funciones y clases auxiliares**

In [6]:
def generate_file_hash(file_path: str) -> str:
    """
    Genera el hash MD5 del contenido de un archivo para verificar su integridad,
    asegurando que no haya sido corrompido o modificado.
    Hash MD5: 128 bits (32 caracteres hexadecimales)

    Args:
        file_path (str): Ruta al archivo

    Returns:
        str: Hash MD5 del archivo (en formato hexadecimal) o cadena vacía en caso de error
    """
    # crea objeto hash MD5
    hash_md5 = hashlib.md5()

    try:
        # abre archivo en modo binario
        with open(file_path, "rb") as f:
            # lee bloques de 4096 bytes, se detiene cuando f.read() devuele b"" (fin de archivo)
            for chunk in iter(lambda: f.read(4096), b""):
                # actualiza el hash con cada bloque leído
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
    except Exception as e:
        print(f"Error generando hash para {file_path}: {e}")
        return ""

In [7]:
def get_file_size_mb(file_path: str) -> float:
    """
    Obtiene el tamaño de archivo en MB

    Args:
        file_path (str): Ruta al archivo

    Returns:
        float: Tamaño del archivo en MB
    """
    try:
        return os.path.getsize(file_path) / (1024 * 1024)
    except:
        return 0.0

In [8]:
def log_ingestion_event(event_type: str, details: Dict):
    """
    Registra eventos de ingesta en archivo de log

    Args:
        event_type (str): Tipo de evento
        details (Dict): Detalles del evento
    """
    # diccionario de fecha actual, tipo de evento recibido y detalles del evento
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "event_type": event_type,
        "details": details
    }

    # ruta del archivo log (un JSON por línea)
    log_file = f"{METADATA_DIR}/ingestion_log.jsonl"
    # abre el archivo para añadir líneas
    with open(log_file, "a") as f:
        # convierte diccionario a JSON, añade salto de línea y lo escribe al final del archivo
        f.write(json.dumps(log_entry) + "\n")

In [9]:
class NYCTaxiDownloadManager:
    """
    Clase para gestionar la descarga de archivos de NYC Taxi
    """
    def __init__(self, config: Dict):
        """
        Inicializa el DownloadManager con configuración

        Args:
            config (Dict): Configuración de la descarga
        """
        # guarda la configuración
        self.config = config
        # estadísticas de descarga
        self.download_stats = []

    def generate_url(self, year: int, month: int) -> str:
        """
        Genera URL de descarga para un mes específico

        Args:
            year (int): Año del mes
            month (int): Mes del año

        Returns:
            str: URL de descarga
        """
        # nombre del archivo
        filename = f"{self.config['data_type']}_{year}-{month:02d}.{self.config['file_format']}"
        # URL de descarga
        return f"{self.config['base_url']}/{filename}"

    def generate_local_path(self, year: int, month: int) -> str:
        """
        Genera path local para guardar archivo

        Args:
            year (int): Año del archivo
            month (int): Mes del archivo

        Returns:
            str: Path local
        """
        # nombre del archivo
        filename = f"{self.config['data_type']}_{year}-{month:02d}.{self.config['file_format']}"
        # ruta local
        return f"{RAW_DIR}/{filename}"

    def file_exists_and_valid(self, file_path: str) -> bool:
        """
        Comprueba si archivo existe y tiene tamaño válido (al menos 10 MB)

        Args:
            file_path (str): Ruta al archivo

        Returns:
            bool: True si existe y tiene tamaño válido, False en caso contrario
        """
        # comprueba si existe el archvio
        if not os.path.exists(file_path):
            return False

        # obtiene el tamaño del archivo
        size_mb = get_file_size_mb(file_path)
        # comprueba que tenga al menos 10 MB
        return size_mb >= 10.0

    def download_file(self, year: int, month: int) -> Dict:
        """
        Descarga un archivo específico con reintentos

        Args:
            year (int): Año del archivo
            month (int): Mes del archivo

        Returns:
            Dict: Información de la descarga
        """
        # URL de descarga
        url = self.generate_url(year, month)
        # directorio local
        local_path = self.generate_local_path(year, month)

        # registro de detalles de la descarga
        download_info = {
            "year": year,
            "month": month,
            "url": url,
            "local_path": local_path,
            "success": False,
            "error": None,
            "size_mb": 0,
            "hash": "",
            "download_time_seconds": 0
        }

        # comprueba si el fichero existe en local y es válido
        if self.file_exists_and_valid(local_path):
            # actualiza el registro
            download_info.update({
                "success": True,
                "size_mb": get_file_size_mb(local_path),
                "hash": generate_file_hash(local_path),
                "skipped": True
            })
            print(f"✓ Fichero existe: {year}-{month:02d} ({download_info['size_mb']:.1f} MB)")
            # devuelve información de la descarga
            return download_info

        ## descarga con reintentos

        # intenta hasta 'max_retries' veces la descarga
        for attempt in range(self.config['max_retries']):
            try:
                print(f"📥 Descargando {year}-{month:02d} (intento ",
                 f"{attempt + 1}/{self.config['max_retries']})")

                # inicia descarga con timeout; lanza error si hay fallo
                start_time = datetime.now()
                response = requests.get(url, timeout=self.config['timeout_seconds'])
                response.raise_for_status()

                # guarda el contenido en el archivo local
                with open(local_path, 'wb') as f:
                    f.write(response.content)

                # calcula la duración de la descarga
                end_time = datetime.now()
                download_time = (end_time - start_time).total_seconds()

                # verifica la descarga
                if self.file_exists_and_valid(local_path):
                    # actualiza el registro
                    download_info.update({
                        "success": True,
                        "size_mb": get_file_size_mb(local_path),
                        "hash": generate_file_hash(local_path),
                        "download_time_seconds": download_time,
                        "attempt": attempt + 1
                    })
                    print(f"✅ Descargado: {year}-{month:02d} ({download_info['size_mb']:.1f} MB)")
                    break
                else:
                    raise Exception("Archivo descargado inválido")

            except Exception as e:
                # hay algún error
                download_info["error"] = str(e)
                print(f"❌ Error en intento {attempt + 1}: {e}")

                if attempt < self.config['max_retries'] - 1:
                    # si no es el último intento, espera 5 seg antes de reintentar
                    print("⏳ Reintentando en 5 segundos...")
                    import time
                    time.sleep(5)

        # devuelve la información de la descarga
        return download_info

    def download_all(self) -> List[Dict]:
        """
        Descarga todos los archivos definidos en la configuración

        Returns:
            List[Dict]: Lista de estadísticas de descarga
        """
        print("=== INICIANDO DESCARGA MASIVA ===")

        # recorre todos los meses a descargar
        for month_config in self.config['months_to_ingest']:
            # descarga el mes actual
            download_result = self.download_file(month_config['year'], month_config['month'])
            # guarda el resultado
            self.download_stats.append(download_result)

            # log del evento
            log_ingestion_event("download", download_result)

        return self.download_stats

In [10]:
class DataSchemaValidator:
    """
    Valida el esquema (estructura y tipos de columnas) de un Dataframe
    """

    def __init__(self):
        """
        Esquema esperado para NYC Taxi data
        """
        self.expected_schema = StructType([
            StructField("VendorID", LongType(), True),
            StructField("tpep_pickup_datetime", TimestampType(), True),
            StructField("tpep_dropoff_datetime", TimestampType(), True),
            StructField("passenger_count", DoubleType(), True),
            StructField("trip_distance", DoubleType(), True),
            StructField("RatecodeID", DoubleType(), True),
            StructField("store_and_fwd_flag", StringType(), True),
            StructField("PULocationID", LongType(), True),
            StructField("DOLocationID", LongType(), True),
            StructField("payment_type", LongType(), True),
            StructField("fare_amount", DoubleType(), True),
            StructField("extra", DoubleType(), True),
            StructField("mta_tax", DoubleType(), True),
            StructField("tip_amount", DoubleType(), True),
            StructField("tolls_amount", DoubleType(), True),
            StructField("improvement_surcharge", DoubleType(), True),
            StructField("total_amount", DoubleType(), True),
            StructField("congestion_surcharge", DoubleType(), True),
            StructField("airport_fee", DoubleType(), True)
        ])

    def validate_schema(self, df, file_info: Dict) -> Dict:
        """
        Valida el esquema de un DataFrame

        Args:
            df (DataFrame): DataFrame a validar
            file_info (Dict): Información del archivo (año y mes)

        Returns:
            Dict: Resultado de la validación
        """
        # inicializa resultados de la validación
        validation_result = {
            "file": f"{file_info['year']}-{file_info['month']:02d}",
            "schema_valid": True,
            "missing_columns": [],
            "extra_columns": [],
            "type_mismatches": []
        }

        # columnas del dataframe y columnas esperadas
        actual_columns = set(col.lower() for col in df.columns)
        expected_columns = set(field.name.lower() for field in self.expected_schema.fields)

        # columnas faltantes
        validation_result["missing_columns"] = list(expected_columns - actual_columns)

        # columnas de más
        validation_result["extra_columns"] = list(actual_columns - expected_columns)

        ## verifica los tipos de datos

        # crea un mapeo nombre_columna_lowercase -> nombre_original
        colname_map = {col.lower(): col for col in df.columns}

        # recorre las columnas esperadas
        for field in self.expected_schema.fields:

            # comprueba que la columna exista en el dataframe
            field_lower = field.name.lower()
            if field_lower in colname_map:

                # obtiene el tipo en el dataframe y en el esperado
                actual_col = colname_map[field_lower]
                actual_type = dict(df.dtypes)[actual_col]
                expected_type = str(field.dataType).lower()

                # compara los tipos
                if not self._types_compatible(actual_type, expected_type):
                    validation_result["type_mismatches"].append({
                        "column": field.name,
                        "expected": expected_type,
                        "actual": actual_type
                    })

        # marca si el esquema del dataframe es válido
        validation_result["schema_valid"] = (
            len(validation_result["missing_columns"]) == 0 and
            len(validation_result["type_mismatches"]) == 0
        )

        # devuelve el resultado de la validación
        return validation_result

    def _types_compatible(self, actual: str, expected: str) -> bool:
        """
        Verifica si los tipos son compatibles

        Args:
            actual (str): Tipo actual (del dataframe actual)
            expected (str): Tipo esperado

        Returns:
            bool: True si son compatibles, False en caso contrario
        """
        # mapeo de tipos de PySpark y otros comunes que puede tener el dataframe actual
        # biblioteca { tipo PySpark: tipos compatibles }
        '''
        IMPORTANTE: para estos datasets se mapea bigint con DoubleType porque
        columnas como passenger_count y RatecodeID pueden venir como entero aunque sean
        enteros pequeños, y no habrá problema en convertir en DoubleType más adelante.
        '''
        type_mappings = {
            "longtype": ["long", "bigint", "int"],
            "long": ["long", "bigint", "int"],
            "bigint": ["long", "bigint", "int"],
            "integer": ["long", "bigint", "int"],
            "double": ["double", "float", "long", "bigint", "int"],
            "doubletype": ["double", "float", "long", "bigint", "int"],
            "string": ["string"],
            "stringtype": ["string"],
            "timestamp": ["timestamp", "datetime"],
            "timestamptype": ["timestamp", "datetime"]
        }

        # convierte tipo del dataframe actual a minúsculas
        actual = actual.lower()
        # convierte tipo esperado a minúsculas
        expected = expected.lower()

        # recorre el mapeo { tipo PySpark: tipos compatibles }
        for expected_base, compatible_types in type_mappings.items():

            # comprueba si el tipo PySpark está contenido en el tipo esperado
            # ej: expected = "bigint unsigned", expected_base = "bigint"
            if expected_base in expected:

                # comprueba si el tipo actual es compatible con alguno de los tipos esperados:
                # recorre la lista de tipos compatibles, devuelve True si el tipo actual
                # está contenido en los tipos compatibles
                return any(comp_type in actual for comp_type in compatible_types)

        # si no se encontró ningún tipo base en expected, comprueba si actual y expected son iguales
        return actual == expected

In [11]:
# v2
class DataContentValidator:
    """
    Valida el contenido de un DataFrame de NYC Taxi comprobando valores nulos,
    datos inválidos, outliers y coherencia temporal, devolviendo estadísticas resumidas.
    """

    def __init__(self, max_distance: float = 500.0, max_passengers: int = 8):
        """
        Inicializa el validador con parámetros máximos de distancia y número de pasajeros
        """
        self.max_distance = max_distance
        self.max_passengers = max_passengers

    def _get_basic_quality_metrics(self, df: DataFrame, get_col_name) -> Dict[str, Any]:
        """
        Calcula agregados de valores nulos y registros con datos inválidos o extremos

        Args:
            df (DataFrame): DataFrame a validar
            get_col_name (callable): Función para obtener el nombre real de una columna

        Returns:
            Dict[str, Any]: Diccionario con los resultados de la agregación
        """

        # contador de nulos por columnas
        nulls = [
            F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f"null_{c}")
            for c in df.columns
        ]

        # nombres reales de las columnas clave
        trip_distance_col = get_col_name("trip_distance")
        fare_amount_col = get_col_name("fare_amount")
        passenger_count_col = get_col_name("passenger_count")

        # contador de registros con valores inválidos/extremos
        quality = [
            F.sum((F.col(trip_distance_col) <= 0).cast("int")).alias("bad_dist"),
            F.sum((F.col(fare_amount_col) <= 0).cast("int")).alias("bad_fare"),
            F.sum((
                (F.col(passenger_count_col) <= 0) | (F.col(passenger_count_col) > self.max_passengers)
                ).cast("int")).alias("bad_pax"),
            F.sum((F.col(trip_distance_col) > self.max_distance).cast("int")).alias("extreme_dist")
        ]

        # ejecuta la agregación
        row = df.select(*(nulls + quality)).first()

        # devuelve resultado como diccionario
        return row.asDict()

    def _get_null_percentages(self, stats: Dict[str, Any], cols: list, total: int) -> Dict[str, float]:
        """
        Calcula el porcentaje de valores nulos por columna

        Args:
            stats (Dict[str, Any]): Diccionario con los resultados de la agregación
            cols (list): Lista de nombres de columna
            total (int): Total de registros

        Returns:
            Dict[str, float]: Diccionario con los porcentajes
        """
        return {
            col: round((stats.get(f"null_{col}", 0) or 0) / total * 100, 2)
            for col in cols
        }

    def _get_data_quality_issues(self, stats: Dict[str, Any], total: int) -> Dict[str, float]:
        """
        Calcula el porcentaje de registros con datos inválidos
        - distancia
        - tarifa
        - número de pasajeros
        - outliers

        Args:
            stats (Dict[str, Any]): Diccionario con los resultados de la agregación
            total (int): Total de registros

        Returns:
            Dict[str, float]: Diccionario con los porcentajes
        """

        return {
            "invalid_distance_pct": round((stats.get("bad_dist", 0) or 0) / total * 100, 2),
            "invalid_fare_pct": round((stats.get("bad_fare", 0) or 0) / total * 100, 2),
            "invalid_passengers_pct": round((stats.get("bad_pax", 0) or 0) / total * 100, 2),
            "extreme_distance_pct": round((stats.get("extreme_dist", 0) or 0) / total * 100, 2)
        }

    def _get_temporal_stats(self, df: DataFrame, get_col_name, total: int) -> Dict[str, Any]:
        """
        Evalúa coherencia temporal entre pickup (inicio del viaje) y dropoff (final),
        y extrae fechas mínimas y máximas

        Args:
            df (DataFrame): DataFrame a validar
            get_col_name (callable): Función para obtener el nombre real de una columna
            total (int): Total de registros

        Returns:
            Dict[str, Any]: Diccionario con los resultados de la agregación
        """

        # nombres reales de las columnas clave
        pickup_col = get_col_name("tpep_pickup_datetime")
        dropoff_col = get_col_name("tpep_dropoff_datetime")

        # comprueba si las columnas temporales existen
        if pickup_col is None or dropoff_col is None:
            # columnas temporales no existen
            print(f"Advertencia: Columnas temporales no encontradas. pickup_col: {pickup_col}, ",
                  f"dropoff_col: {dropoff_col}")
            return {
                "inverted_dates_pct": 0.0,
                "min_pickup": "N/A",
                "max_pickup": "N/A",
                "min_dropoff": "N/A",
                "max_dropoff": "N/A"
            }

        # columnas temporales sí existen
        try:
            row = df.select(
                F.sum((F.col(dropoff_col) < F.col(pickup_col)).cast("int")).alias("inverted_dates"),
                F.min(F.col(pickup_col)).alias("min_pu"),
                F.max(F.col(pickup_col)).alias("max_pu"),
                F.min(F.col(dropoff_col)).alias("min_do"),
                F.max(F.col(dropoff_col)).alias("max_do")
            ).first()

            return {
                "inverted_dates_pct": round((row["inverted_dates"] or 0) / total * 100, 2),
                "min_pickup": str(row["min_pu"]) if row["min_pu"] else "N/A",
                "max_pickup": str(row["max_pu"]) if row["max_pu"] else "N/A",
                "min_dropoff": str(row["min_do"]) if row["min_do"] else "N/A",
                "max_dropoff": str(row["max_do"]) if row["max_do"] else "N/A"
            }

        except Exception as e:
            print(f"Error en agregaciones temporales: {e}")
            return {
                "inverted_dates": 0,
                "min_pickup": "N/A",
                "max_pickup": "N/A",
                "min_dropoff": "N/A",
                "max_dropoff": "N/A"
            }

    def validate_content(self, df: DataFrame, file_info: Dict[str, int]) -> Dict[str, Any]:
        """
        Ejecuta todas las validaciones sobre el Dataframe y devuelve un resumen estructurado

        Args:
            df (DataFrame): DataFrame a validar
            file_info (Dict): Información del archivo (año y mes)

        Returns:
            Dict: Resultado de la validación
        """

        # inicializa el resultado
        result = {
            "file": f"{file_info['year']}-{file_info['month']:02d}",
            "total_records": 0,
            "null_percentages": {},
            "data_quality_issues": {},
            "temporal_validation": {}
        }

        try:
            # contador de registros
            result["total_records"] = df.count()

            # comprueba que haya registros
            if result["total_records"] == 0:
                result["status"] = "empty"
                result["data_quality_issues"]["empty_dataset"] = True
                return result

            # mapeo de nombres de columnas ignorando mayúsculas/minúsculas
            colname_map = {col.lower(): col for col in df.columns}
            def get_col_name(normalized_name: str) -> Optional[str]:
                return colname_map.get(normalized_name.lower())

            ## agregaciones de calidad de datos

            # identifica los nombres reales de las columnas clave
            required = ["trip_distance", "fare_amount", "passenger_count"]
            missing = [col for col in required if get_col_name(col) is None]
            if missing:
                result["status"] = "error"
                result["error"] = f"Faltan columnas requeridas: {missing}"
                return result

            basic_stats = self._get_basic_quality_metrics(df, get_col_name)
            result["null_percentages"] = self._get_null_percentages(
                basic_stats, df.columns, result["total_records"]
            )
            result["data_quality_issues"] = self._get_data_quality_issues(
                basic_stats, result["total_records"]
            )

            temporal_stats = self._get_temporal_stats(df, get_col_name, result["total_records"])
            result["temporal_validation"] = temporal_stats

        except Exception as e:
            result["validation_error"] = str(e)
            print(f"Error en validación: {e}")

        return result

**Ingesta de Datos**

In [12]:
# configuración de la ingesta de datos

'''
Datos de NYC Taxi desde la fuente oficial
months_to_ingest: lista de diccionarios, cada uno con el año y mes que se quieren ingerir
Máximo 3 reintentos
Tiempo máximo de espera de 5 minutos (300 seg.)
'''

INGESTION_CONFIG = {
    "base_url": "https://d37ci6vzurychx.cloudfront.net/trip-data",
    "data_type": "yellow_tripdata",
    "file_format": "parquet",
    "months_to_ingest": [
        {"year": 2023, "month": 1},
        {"year": 2023, "month": 2},
        {"year": 2023, "month": 3}
    ],
    "max_retries": 3,
    "timeout_seconds": 300
}

print("=== CONFIGURACIÓN DE INGESTA ===")
print(f"Proyecto: {PROJECT_ROOT}")
print(f"Meses a ingestar: {len(INGESTION_CONFIG['months_to_ingest'])}")
for month_config in INGESTION_CONFIG['months_to_ingest']:
    print(f"  - {month_config['year']}-{month_config['month']:02d}")

=== CONFIGURACIÓN DE INGESTA ===
Proyecto: /content/drive/MyDrive/taxi_project
Meses a ingestar: 3
  - 2023-01
  - 2023-02
  - 2023-03


In [13]:
# crea una instancia del gestor de descargas
download_manager = NYCTaxiDownloadManager(INGESTION_CONFIG)

In [14]:
# descarga todos los archivos definidos en la configuración de la ingesta
download_results = download_manager.download_all()

=== INICIANDO DESCARGA MASIVA ===
✓ Fichero existe: 2023-01 (45.5 MB)
📥 Descargando 2023-02 (intento  1/3)
✅ Descargado: 2023-02 (45.5 MB)
📥 Descargando 2023-03 (intento  1/3)
✅ Descargado: 2023-03 (53.5 MB)


In [15]:
# descargas correctas
successful_downloads = [r for r in download_results if r['success']]

print(f"✅ Descargas correctas: {len(successful_downloads)}")
if successful_downloads:

    # suma el tamaño en MB de los archivos descargados correctamente
    total_size = sum(r['size_mb'] for r in successful_downloads)
    print(f"📊 Tamaño total correctas: {total_size:.1f} MB")

print()

# descargas fallidas
failed_downloads = [r for r in download_results if not r['success']]

print(f"❌ Descargas fallidas: {len(failed_downloads)}")
if failed_downloads:
    # muestra información de los archivos fallidos
    for failed in failed_downloads:
        print(f"  - {failed['year']}-{failed['month']:02d}: {failed['error']}")

✅ Descargas correctas: 3
📊 Tamaño total correctas: 144.5 MB

❌ Descargas fallidas: 0


In [16]:
# valida los archivos descargados
print("=== VALIDANDO ARCHIVOS DESCARGADOS ===")

# crea instancia de clase de validación de esquema
schema_validator = DataSchemaValidator()

# crea instancia de clase de validación de datos
MAX_DISTANCE = 300
MAX_PASSENGERS = 8
content_validator = DataContentValidator(max_distance=MAX_DISTANCE, max_passengers=MAX_PASSENGERS)

# inicializa la lista de resultados
validation_results = []

# recorre los archivos descargados correctamente
for download_result in successful_downloads:

    print(f"\n🔍 Validando {download_result['year']}-{download_result['month']:02d}...")

    try:
        # carga el archivo en un Dataframe de Spark
        ds = spark.read.parquet(download_result['local_path'])

        # valida el esquema
        schema_validation = schema_validator.validate_schema(ds, download_result)

        # valida el contenido
        content_validation = content_validator.validate_content(ds, download_result)

        # combina ambos resultados
        combined_validation = {
            **download_result,
            "schema_validation": schema_validation,
            "content_validation": content_validation
        }

        # agrega el resultado combinado a la lista de resultados
        validation_results.append(combined_validation)

        # guarda el log de la validación
        log_ingestion_event("validation", combined_validation)

        print(f"  📊  Registros: {content_validation['total_records']:,}")

        # muestra un resumen de la validación
        if schema_validation["schema_valid"]:
            print(f"  ✅  Esquema válido")
        else:
            print(f"  ❌  Esquema no válido")
            display(schema_validation)

        # muestra problemas de calidad de los datos
        quality_issues = content_validation.get('data_quality_issues', {})
        for issue, percentage in quality_issues.items():

            # sólo muestra los problemas de calidad si son > 0%
            MIN_PERCENTAGE = 0

            if percentage > MIN_PERCENTAGE:
              print(f"  ⚠️  {issue}: {percentage}%")

    except Exception as e:
        # ha ocurrido algún error
        print(f"  ❌ Error validando: {e}")
        validation_results.append({
            **download_result,
            "validation_error": str(e)
        })

=== VALIDANDO ARCHIVOS DESCARGADOS ===

🔍 Validando 2023-01...
  📊  Registros: 3,066,766
  ✅  Esquema válido
  ⚠️  invalid_distance_pct: 1.5%
  ⚠️  invalid_fare_pct: 0.85%
  ⚠️  invalid_passengers_pct: 1.67%

🔍 Validando 2023-02...
  📊  Registros: 2,913,955
  ✅  Esquema válido
  ⚠️  invalid_distance_pct: 1.41%
  ⚠️  invalid_fare_pct: 0.89%
  ⚠️  invalid_passengers_pct: 1.62%

🔍 Validando 2023-03...
  📊  Registros: 3,403,766
  ✅  Esquema válido
  ⚠️  invalid_distance_pct: 1.43%
  ⚠️  invalid_fare_pct: 0.91%
  ⚠️  invalid_passengers_pct: 1.71%


**Bronze Layer**

Transforma los archivos validados en una capa Bronze: capa persistente de datos en crudo con metadatos, guardados en Parket, particionados y preparados para etapas posteriores.

In [17]:
# crea la capa Bronze

print("=== CREANDO BRONZE LAYER ===")

# inicializa lista de resultados
bronze_datasets = []

# recorre cada resultado de la validación
for validation_result in validation_results:

    # procesa sólo los archivos descargados correctamente y con esquema válido
    if (validation_result.get(
        'success', False
        ) and validation_result.get(
            'schema_validation', {}
        ).get('schema_valid', False)
        ):

        # año y mes del Dataset
        year = validation_result['year']
        month = validation_result['month']

        print(f"📦 Procesando {year}-{month:02d} para Bronze Layer...")

        try:
            # carga el archivo desde disco
            ds = spark.read.parquet(validation_result['local_path'])

            # añade los metadatos de ingesta
            ds_bronze = ds.withColumn("ingestion_timestamp", current_timestamp()) \
                         .withColumn("source_file", lit(f"{year}-{month:02d}")) \
                         .withColumn("ingestion_year", lit(year)) \
                         .withColumn("ingestion_month", lit(month))

            ## guarda el Dataframe

            # append: añade datos sin borrar lo anterior
            # particionado por año y mes de ingesta
            # compresión snappy
            # formato parquet en la ruta indicada
            ds_bronze.write \
                .mode("append") \
                .partitionBy("ingestion_year", "ingestion_month") \
                .option("compression", "snappy") \
                .parquet(BRONZE_PATH)

            # añade registro con información del dataset procesado
            bronze_datasets.append({
                "year": year,
                "month": month,
                "records": validation_result['content_validation']['total_records'],
                "bronze_path": BRONZE_PATH
            })

            print("  ✅ Guardado en Bronze: ",
             f"{validation_result['content_validation']['total_records']:,} registros")

        except Exception as e:
            # ha ocurrido algún error
            print(f"  ❌ Error guardando en Bronze: {e}")

    else:
      print("sin datos")

=== CREANDO BRONZE LAYER ===
📦 Procesando 2023-01 para Bronze Layer...
  ✅ Guardado en Bronze:  3,066,766 registros
📦 Procesando 2023-02 para Bronze Layer...
  ✅ Guardado en Bronze:  2,913,955 registros
📦 Procesando 2023-03 para Bronze Layer...
  ✅ Guardado en Bronze:  3,403,766 registros


In [18]:
# verifica la capa Bronze y genera un archivo JSON con la información de ingesta

print("=== VERIFICANDO BRONZE LAYER ===")

try:
    # carga los datos de la capteta de la capa Bronze
    bronze_df = spark.read.parquet(f"{BRONZE_DIR}/taxi_data")
    print(f"✅ Bronze Layer creado exitosamente")


    # número total de registros
    total_bronze_records = bronze_df.count()
    print(f"📊 Total de registros en Bronze: {total_bronze_records:,}")

    # distribución por año y mes de ingesta con conteo de registros por cada partición
    month_distribution = bronze_df.groupBy("ingestion_year", "ingestion_month") \
                                 .count() \
                                 .orderBy("ingestion_year", "ingestion_month") \
                                 .collect()

    # muestra la distribución por mes
    print("\n📅 Distribución por mes:")
    for row in month_distribution:
        print(f"  {row['ingestion_year']}-{row['ingestion_month']:02d}: {row['count']:,} registros")

    # crea diccionario de metadatos
    bronze_metadata = {
        "creation_timestamp": datetime.now().isoformat(),
        "total_records": total_bronze_records,
        "datasets_included": bronze_datasets,
        "validation_summary": {
            "total_files_processed": len(validation_results),
            "successful_validations": len([
                v for v in validation_results if v.get('schema_validation', {}).get('schema_valid', False)
                ]),
            "total_size_mb": sum(r['size_mb'] for r in successful_downloads)
        }
    }

    # guarda los metadatos en un archivo JSON
    with open(f"{METADATA_DIR}/bronze_layer_metadata.json", "w") as f:
        json.dump(bronze_metadata, f, indent=2)

    print(f"\n💾 Metadatos guardados en: {METADATA_DIR}/bronze_layer_metadata.json")

except Exception as e:
    print(f"❌ Error verificando Bronze Layer: {e}")

    # muestra detalles del error
    import traceback
    traceback.print_exc()

=== VERIFICANDO BRONZE LAYER ===
✅ Bronze Layer creado exitosamente
📊 Total de registros en Bronze: 9,384,487

📅 Distribución por mes:
  2023-01: 3,066,766 registros
  2023-02: 2,913,955 registros
  2023-03: 3,403,766 registros

💾 Metadatos guardados en: /content/drive/MyDrive/taxi_project/metadata/bronze_layer_metadata.json


In [19]:
# cierra Spark
spark.stop()
print("🔌 Sesión Spark cerrada")

🔌 Sesión Spark cerrada
